In [1]:
from UNet import Unet
import utils
import engine
from learning_rate_range_test import LRTest

import os
import numpy as np
import matplotlib.pyplot as plt



import albumentations as A
import gc
from tqdm.notebook import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim


c:\Users\CHIRANJIT\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model=Unet(channels=[3, 64, 128, 256, 512, 1024], no_classes=150)
x   = torch.Tensor(np.random.rand(2, 3, 572, 572))
y=model(x)
print(y.shape)

torch.Size([2, 150, 388, 388])


In [ ]:
datadir  = './ADE20K/images/training/'
maskdir = './ADE20K/annotations/training/'
val_datadir  = './ADE20K/images/validation/'
val_maskdir = './ADE20K/annotations/validation/'




In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

class ADE20KDataset(Dataset):
    def __init__(self, image_dir, mask_dir, image_size=(256, 256)):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_files = sorted(os.listdir(image_dir))
        self.mask_files = sorted(os.listdir(mask_dir))
        self.image_size = image_size
        self.image_transform = transforms.Compose([
            transforms.Resize(self.image_size),
            transforms.RandomHorizontalFlip(),
            transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        image = Image.open(os.path.join(self.image_dir, self.image_files[idx])).convert("RGB")
        mask = Image.open(os.path.join(self.mask_dir, self.mask_files[idx]))
        mask = mask.resize(self.image_size, Image.NEAREST)

        image = self.image_transform(image)
        mask = torch.from_numpy(np.array(mask, dtype=np.int64))

        return image, mask
